In [40]:
import pprint
pp = pprint.PrettyPrinter(indent=4, width=200, depth=None, stream=None, compact=False, sort_dicts=False)

In [6]:
from tensorflow.keras.applications import EfficientNetV2B1
model = EfficientNetV2B1(include_top=False, pooling='avg', weights='imagenet')
model.trainable = False

In [7]:
print(model.get_layer('avg_pool'))

In [ ]:
#model.summary()

In [8]:
from keras.models import Model
my_model= Model(inputs=model.input, outputs=model.get_layer('avg_pool').output)

In [ ]:
#my_model.summary()

In [2]:
import cv2
import numpy as np
import pandas as pd
import os
from matplotlib.pyplot import imread
from matplotlib.pyplot import imshow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import decode_predictions, preprocess_input


In [5]:
df = pd.read_csv("captions_hu.txt",sep="|")
df['id'] = [id_ for id_ in range(df.shape[0] // 5) for _ in range(5)]
df.to_csv("captions.csv", index=False)
df = pd.read_csv("captions.csv")
df.head()
df.tail()

,image,caption,id
40450,997722733_0cb5439472.jpg,Egy rózsaszín inges férfi felmászik egy szikla...,8090
40451,997722733_0cb5439472.jpg,Egy férfi sziklát mászik a magasban a levegőben.,8090
40452,997722733_0cb5439472.jpg,Egy piros inges személy felkapaszkodik egy seg...,8090
40453,997722733_0cb5439472.jpg,Egy sziklamászó piros ingben.,8090
40454,997722733_0cb5439472.jpg,Egy sziklamászó egy sziklamászófalon gyakorol.,8090


In [10]:
path = 'Images/'
filenames = os.listdir('Images')
#print(filenames)
im_size = 240

images = []

   
for f in filenames:
    #print(path + f)
    img = cv2.imread(path + f)
    img = cv2.resize(img, (im_size, im_size))
    images.append(img)


In [11]:
images = np.array(images)

images = images.astype('float32') / 255.0
images.shape

(8091, 224, 224, 3)

In [ ]:
outputs = []
for i in range(len(images)):
    print(i, end="\r")
    outputs.append(my_model(images[i]))

#pp.pprint(outputs)    

In [30]:
outputs_np = np.array(outputs, dtype="float32")
print(outputs_np.shape)
outputs_np.tofile("img_embs_8k.bin")

(8091, 1, 1280)


In [32]:
input = np.fromfile('img_embs_8k.bin', dtype='float32')
input = np.reshape(input,(8091, 1, 1280),)
print(input.shape)

(8091, 1, 1280)


In [33]:
print(numpy.array_equal(outputs_np,input))

True


#old

img_path='img.jpg'
my_img=imread(img_path)
#imshow(my_img)

SIZE = 224

img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
h = SIZE # img.shape[0] - h - fixed
w = round(img.shape[1]*(h/img.shape[0])) # keeps the ratio
new_img = cv2.resize(img, (w,h))
w_start=round((w-h)/2)
w_end=w_start+SIZE
crop_img = new_img[0:SIZE, w_start:w_end]
imshow(crop_img)

x = np.expand_dims(crop_img,axis=0)
x = preprocess_input(x)

print('input image shape:', x.shape)

#old

img_path='img.jpg'
my_img=imread(img_path)
imshow(my_img)

#old

outputs = my_model(my_img)

#old

pp.pprint(outputs)